# TP3 - Detectionsegmentation et estimation de la pose des humains.

In [1]:
! pip install ultralytics
! pip install numpy
! pip install cv2
! pip install torch
! pip install playsound==1.2.2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


## Partie 1 - Manipulations de base avec YOLO11 pour la détection dans une image.


In [ ]:
from ultralytics import YOLO

# detect animal
model = YOLO("models/yolo11n.pt")
results = model("img/animal/animal2.jpg")

# object segmentation 
# model = YOLO("yolo11n-seg.pt")
# results = model("img/segmentation.jpg")

# detect person pose
# model = YOLO("yolo11n-pose.pt")
# results = model("img/pose/pose2.jpg")

# oriented detection
# model = YOLO("yolo11n-obb.pt")
# results = model("img/orientedDetection.jpg")

# classification
# model = YOLO("yolo11n-cls.pt")
# results = model("img/pose/pose3.webp")

print(results[0])



image 1/1 c:\Users\Isio\Documents\TraitementImage\TP3\img\animal\animal2.jpg: 448x640 1 bird, 100.0ms
Speed: 2.9ms preprocess, 100.0ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoo

In [12]:
import os
print(os.path.dirname('__file__'))

In [23]:
! pip install playsound==1.2.2

  Using cached playsound-1.2.2-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached playsound-1.2.2-py2.py3-none-any.whl (6.0 kB)


## Partie 2 - Application de suivi avec YOLO11

In [ ]:
from ultralytics import YOLO

import numpy as np
import cv2 as cv
import os


from time import time
from playsound import playsound

timeForAlarm = 5

classesToDetect = {
    # 39: 'bottle', 
    # 41: 'cup', 
    # 42: 'fork', 
    # 43: 'knife', 
    # 44: 'spoon', 
    # 45: 'bowl', 
    67: 'cell phone', 
    # 73: 'book', 
    # 76: 'scissors', 
    # 79: 'toothbrush',
}

cap = cv.VideoCapture(0)
if (not cap.isOpened()):
    print("Error while opening the camera.")
    exit(1)
framerate = 60
period = int(1000/framerate)
    
model = YOLO(model="models/yolo11s.pt")

detectionTime = time()
detectedClass = -1
detectedId = -1

while (True):
    ret, frame = cap.read()
    
    if (not ret):
        print("Error while fetching the image from the camera.")
        break

    # treat the image
    results = model.track(frame, classes=[_ for _ in classesToDetect.keys()], tracker="bytetrack.yaml")

    labels = results[0].names
    
    # no objets of interest detected
    if len(results[0]) == 0:
        detectedClass = -1
        detectedId = -1
    
    for detectedObj in results[0]:
        boxes = detectedObj.boxes

        objClass = int(boxes.cls[0])
        objClassStr = labels[objClass]
        classConf = boxes.conf.tolist()[0]
        
        if boxes.id == None: continue
        
        trackingId = int(boxes.id.tolist()[0])

        # detected object of interest, mark the first one for tracking purpose
        if detectedClass == -1:
            detectionTime = time()
            detectedClass = objClass
            detectedId = trackingId

        # bounding box coordinates 
        x1, y1, x2, y2 = [int(i) for i in boxes.xyxy[0].tolist()]
        
        box_color = (255, 255, 255)
        text_color = (0, 0, 0)
        
        # object detected for long enough, play sound and switch bounding box and label color
        timeElapsed = time() - detectionTime     
        if timeElapsed > timeForAlarm: 
            box_color = (0, 0, 255)
            text_color = (255, 255, 255)
            
            # playsound not working
            # playsound(f'{os.path.abspath("./")}\\{"alarm.wav"}', block=False)
            
        label = f"{objClassStr} (id: {trackingId}): {classConf:.2f}"
        textBgLength = len(label)*8 + 20
        
        if trackingId == detectedId:
            # bounding box
            frame = cv.rectangle(frame, (x1, y1), (x2, y2), box_color, 3)
            # background for label
            frame = cv.rectangle(frame, (x1, y1), (x1+textBgLength, y1-30), box_color, -1)
            # label with class, class index and conf
            frame = cv.putText(frame, label, (x1+5, y1-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)
        
    cv.imshow("Camera", frame)
    
    if (cv.waitKey(period) == ord('q')):
        break        
    
cv.destroyAllWindows()
cap.release()
    


0: 480x640 (no detections), 176.3ms
Speed: 3.3ms preprocess, 176.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 173.0ms
Speed: 2.0ms preprocess, 173.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 147.0ms
Speed: 1.0ms preprocess, 147.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 143.0ms
Speed: 1.0ms preprocess, 143.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 201.0ms
Speed: 1.0ms preprocess, 201.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 150.0ms
Speed: 1.0ms preprocess, 150.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 149.0ms
Speed: 1.0ms preprocess, 149.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 151.0ms
Speed: 2.0ms prepr

PlaysoundException: 
    Error 296 for command:
        open "c:\Users\Isio\Documents\TraitementImage\TP3\alarm.wav" alias playsound_0.41817289685129133
    The specified file cannot be played on the specified MCI device.  The file may be corrupt, not in the correct format, or no file handler available for this format.